# Modélisation de l’incertitude — Master 2 MIAGE IA2
## Travaux dirigés N◦5 : Théorie de l’évidence

On va utiliser la théorie de l’évidence pour résoudre un problème de combinaison d’évidence de sources différentes. Pour ce faire, on s’appuyera sur la bibliothèque Python pyds de Thomas Reineking.

# 1. Introduction
La bibliothèque Python pyds de Thomas Reineking peut être téléchargée à partir de la page http://reineking.github.io/pyds/. Regardez le fichier examples.py pour comprendre le fonctionnement de la bibliothèque.

# 2 Le mystère du vol MH370
Le 8 mars 2014, le Boeing 777 du vol Malaysia Airlines 370 (MH370) est porté disparu avec 239 personnes à son bord. L’appareil, qui devait relier Kuala Lumpur à Pékin, après avoir disparu des radars vietnamiens, aurait continué à voler pendant des heures, avant de s’abîmer en mer. Malgré d’intenses recherches, le lieu de sa disparition n’a pas pu être déterminé. Seuls quelques débris pouvant avoir appartenu à l’avion ont été retrouvés, plus d’un an après, sur le littoral africain. Il s’agit, à ce jour, du plus grand mystère de l’histoire de l’aviation civile. Rolls Royce, le constructeur des moteurs de l’avions, aurait reçu des communications ACARS toutes les trente minutes pendant cinq heures après la perte de contact avec l’avion. Sur la base de ces signaux, on a pu estimer que la position de l’appareil devait être située sur la circonférence d’un cercle correspondant à une inclinaison de 30◦par rapport à la verticale de la position du satellite qui les captait.
Enfin, quelques habitants des Maldives auraient rapporté d’avoir vu l’avion, en train de voler à basse altitude, mais ce témoignage est considéré peu fiable.

# 3. Consignes
On se propose d’utiliser la théorie de l’évidence pour aider les recherches de l’épave du vol MH370. On simplifiera le problème en divisant la zone des recherches, qui correspond en gros à l’Océan Indien, en neuf secteurs, suivant le cadrillage montré dans la Figure 1.

In [1]:
from __future__ import print_function
from pyds import MassFunction
from itertools import product

### 1. 
* Le dernier message ACARS suggère que l’avion pouvait être, peu avant de s’abîmer en mer, dans les secteurs :
* NW, N, E, ou SE avec une masse de probabilité de 50%, 
* dans C ou NE avec une masse de 30%, 
* ou dans l’un des secteurs restants avec une masse de 20%. 

Quadrillage de la zone de recherche :



|        |         |        |
| :----: | :-----: | :----: |
| NW     | N       | NE     |
| W      | C       | E      |
| SW     | S       | SE     |



In [8]:
import numpy as np
map_fly = np.array([['NW','N','NE'],
                    ['W','C','E'],
                    ['SW','S','SE']
                    ])
map_fly

array([['NW', 'N', 'NE'],
       ['W', 'C', 'E'],
       ['SW', 'S', 'SE']], dtype='<U2')

In [9]:
print('=== creating mass functions ===')
m1 = MassFunction([({'NW', 'N', 'E','SE'}, 0.5), ({'C', 'NE'}, 0.3), ({'W','SW','S'}, 0.2)]) # using a list of tuples
print('m_1 =', m1)

=== creating mass functions ===
m_1 = {{'SE', 'NW', 'N', 'E'}:0.5; {'C', 'NE'}:0.3; {'SW', 'S', 'W'}:0.2}


2. Le retrouvement de débris sur le littoral africain est une évidence qui pointe au secteur SW (50%) ; cependant, on sait qu’il y a des courants qui vont de l’est à l’ouest dans cette partie de l’océan, donc il se porrait que les débris aient été transporté jusqu’aux côtes africaines des secteurs C ou S (30%) ou E ou SE (20%).

In [10]:
m2 = MassFunction([({'SW'}, 0.5), ({'C', 'S'}, 0.3), ({'E','SE'}, 0.2)]) # using a list of tuples
print('m_2 =', m2)

m_2 = {{'SW'}:0.5; {'C', 'S'}:0.3; {'SE', 'E'}:0.2}


3. Enfin, l’observation douteuse de l’appareil volant à basse altitude aux Maldives est estimée apporter une masse de 10% à une localisation de l’épave dans le secteur W et une masse de 90% partout (y compris le secteur W).

In [11]:
m3 = MassFunction([({'W'}, 0.1), ({'NW','N','NE','W','C','E','SW','S','SE'}, 0.9)]) # using a list of tuples
print('m_3 =', m3)

m_3 = {{'SE', 'C', 'W', 'S', 'NW', 'NE', 'N', 'E', 'SW'}:0.9; {'W'}:0.1}


4. Dans quel secteur faudrait-il concentrer les recherches ? Suggestion : il faudra d’abord combiner les différentes sources d’évidence, puis calculer les probabilités pignistiques.

In [12]:
m4 = m1 & m2 & m3
print(m4)

{{'SE', 'E'}:0.2857142857142857; {'SW'}:0.2857142857142857; {'C'}:0.2571428571428571; {'S'}:0.17142857142857143}



5. En quelle mesure ces conclusions sont sensibles aux hypothèses de distribution des trois masses ? Est-ce que des petites erreurs d’estimation de ces masses (on va dire comprises dans le 5% en plus ou en moins) pourraient faire changer drastiquement le résultat ?
